## Thermal Conductivity Raw Data Fitting
Developed by Henry Nachman

Last Edited: 26 March 2024

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import os, json, shutil

# note : most functions needed for running this notebook can be found in tc_utils.
from tc_utils import *
# Defines the matplotlib backend for plots
# %matplotlib qt5

plots = True # Set to true to reproduce all plots, note this will likely lengthen the time to run the code

First, we need to find where all our RAW data is

In [11]:
path_to_lib = f"{os.getcwd()}\\lib"
mat_directories = [folder for folder in os.listdir(path_to_lib) if not folder.endswith(".md")]

path_to_RAW = dict()

for mat in mat_directories:
    path_to_mat = f"{path_to_lib}\\{mat}"
    raw_str = f"{path_to_mat}\\RAW"
    config_str = f"{path_to_mat}\\config.yaml"
    parent_yaml_str = f"{path_to_mat}\\parent.yaml"
    other_str = f"{path_to_mat}\\OTHERFITS"
    nist_str = f"{path_to_mat}\\NIST"
    source = []
    if os.path.exists(raw_str): # Finds the raw data if it exists.
        path_to_RAW[mat] = raw_str
        source.append("RAW")
    if os.path.exists(other_str): # Finds other fits
        source.append("other")
    if os.path.exists(nist_str): # Finds NIST fit
        source.append("NIST")

    if not os.path.exists(config_str): # Check for existing JSON
        parent = "NA"
    else:
        with open(config_str, 'r') as file:
            mat_config = json.load(file)
        parent = mat_config[0]["parent"]

    yaml_dict = []
    for i in range(len(source)):
        yaml_dict.append({"name":f"{mat}", "parent":f"{parent}", "source":f"{source[i]}"}) # Define JSON dictionary
    yaml_dict = json.dumps(yaml_dict, indent=4)
    with open(config_str, 'w') as file:
        file.write(yaml_dict) # Write to new JSON

    

In [12]:
# Load the JSON
for mat in mat_directories:
    path_to_mat = f"{path_to_lib}\\{mat}"
    raw_str = f"{path_to_mat}\\RAW"
    config_str = f"{path_to_mat}\\config.yaml"

    if os.path.exists(raw_str):
        # other_str = f"{path_to_mat}\\OTHERFITS"
        with open(config_str, 'r') as file:
            mat_config = json.load(file)
        parent = mat_config[0]["parent"]
        if parent != "NA":
            print(mat, "has parent:", parent)
            parent_dir = f"{path_to_lib}\\{parent}"
            if not os.path.exists(parent_dir):
                os.mkdir(parent_dir)
                os.mkdir(f"{parent_dir}\\RAW")
            raw_files = get_datafiles(raw_str)
            for file in raw_files:
                # print(file)
                # try:
                shutil.copy(f"{raw_str}\\{file}", f"{parent_dir}\\RAW\\{file}")
                # except shutil.SameFileError:
                    # print("file already exists")

CFRP_Clearwater has parent: CFRP
Found 5 measurements.
CFRP_DPP has parent: CFRP
Found 2 measurements.
CFRP_Graphlite has parent: CFRP
Found 2 measurements.
Stainless_Steel_304 has parent: Stainless_Steel
Found 8 measurements.
Stainless_Steel_310 has parent: Stainless_Steel
Found 3 measurements.
Stainless_Steel_316 has parent: Stainless_Steel
Found 6 measurements.
Stainless_Steel_321 has parent: Stainless_Steel
Found 6 measurements.


Let's run our fitting code for every material found in the library.

In [13]:
path_to_RAW = dict()
path_to_fits = dict()
path_to_plots = dict()

for mat in mat_directories:
    path_to_mat = f"{path_to_lib}\\{mat}"
    raw_str = f"{path_to_mat}\\RAW"
    fits_str = f"{path_to_mat}\\fits"
    plots_str = f"{path_to_mat}\\plots"
    if os.path.exists(raw_str):
        path_to_RAW[mat] = raw_str
        if not os.path.exists(fits_str):
            os.mkdir(fits_str)
        path_to_fits[mat] = fits_str
        if not os.path.exists(plots_str):
            os.mkdir(plots_str)
        path_to_fits[mat] = fits_str
        path_to_plots[mat] = plots_str

In [14]:
def find_gaps(data_array, threshold):
    # Calculate differences between consecutive elements
    diffs = np.diff(np.sort(data_array))
    # Find indices where differences exceed threshold
    major_gap_indices = np.where(diffs > threshold)[0]

    # Return indices of major gaps
    return major_gap_indices

In [15]:
for mat in  path_to_RAW.keys(): # ["CFRP"]: #
    perc_diff_avgs = np.array([])
    ## First, let's collect the raw data from their csv files
    big_data, data_dict = parse_raw(mat, path_to_RAW[mat], plots=False, weight_const=0.00)
    T, k, koT, weights = [big_data[:,0], big_data[:,1], big_data[:,2], big_data[:,3]]

    gaps = len(find_gaps(T, 100)) > 0
    # print(find_gaps(T, 100))
    maxT, minT = [max(T), min(T)]
    fit_orders = [3,3]
    fit_types = ["k/T", "loglog"]

    lenLow = len(T[T<=20])
    lenHi = len(T[T>=20])

    if lenHi==0:
        print(f"{mat} : Using a low fit")
        low_fit_xs, low_fit = koT_function(T, koT, fit_orders[0], weights)
        hi_fit, hi_fit_xs, erf_loc = [[0], [0], [0]]
        fit_args = dict_combofit(low_fit, low_fit_xs, hi_fit, hi_fit_xs, fit_orders, fit_types, erf_loc)
        perc_diff_low, perc_diff_arr = get_percdiff(T[T<=max(low_fit_xs)],k[T<=max(low_fit_xs)], fit_args)
        fit_args["low_perc_err"] =  perc_diff_low
        fit_args["hi_perc_err"] =  0
        fit_args["combined_perc_err"] =  perc_diff_low
    elif lenLow==0:
        print(f"{mat} : Using a hi fit")
        hi_fit_xs, hi_fit = logk_function(np.log10(T), np.log10(k), fit_orders[1], weights)
        low_fit, low_fit_xs, erf_loc = [[0], [0], [-1]]
        fit_args = dict_combofit(low_fit, low_fit_xs, hi_fit, hi_fit_xs, fit_orders, fit_types, erf_loc)
        perc_diff_hi, perc_diff_arr = get_percdiff(T[T>=min(hi_fit_xs)],k[T>=min(hi_fit_xs)], fit_args)
        fit_args["hi_perc_err"] = perc_diff_hi
        fit_args["low_perc_err"] =  0
        fit_args["combined_perc_err"] =  perc_diff_hi

    else:
        print(f"{mat} : Using a combined fit")# - data exists on both sides of 20K")
        erf_locList = np.linspace(np.sort(T)[0], np.sort(T)[-1], 15)
        for erf_loc in erf_locList:
            dsplit = split_data(big_data, erf_loc)
            lowT, lowT_k, lowT_koT, low_ws, hiT, hiT_k, hiT_koT, hi_ws = dsplit
            # Take a log10 of the high range
            log_hi_T = np.log10(hiT)
            log_hi_k = np.log10(hiT_k)
            
            if (len(lowT)==0):
                low_fit = [0]
                low_fit_xs = [0]
            else:
                low_fit_xs, low_fit = koT_function(lowT, lowT_koT, fit_orders[0], low_ws)
            if (len(hiT)==0):
                hi_fit = [0]
                hi_fit_xs = [0]
            else:
                hi_fit_xs, hi_fit = logk_function(log_hi_T, log_hi_k, fit_orders[1], hi_ws)
            fit_args = dict_combofit(low_fit, low_fit_xs, hi_fit, hi_fit_xs, fit_orders, fit_types, erf_loc)
            ## With the fit complete, let's output a formatted dictionary with the fit parameters
            # output_array = format_combofit(fit_args)
            ## We want to figure out the best location for the split in data, so we will compute the residual of the combined fit
            low_param, hi_param, erf_param = fit_args["low_fit_param"], fit_args["hi_fit_param"], fit_args["combined_fit_param"][-1]
            # kpred = loglog_func(T, low_param, hi_param, erf_param)
            # and append it to the array resVal
            # diff = kpred-k
            # perc_diff_arr = 100*abs(diff/kpred)
            perc_diff_avg, perc_diff_arr = get_percdiff(T, k, fit_args)
            perc_diff_avgs = np.append(perc_diff_avgs, perc_diff_avg)

        # Now that we have found the residuals of the fits for many different split locations, let's choose the best one.    
        erf_locdict = dict(zip(erf_locList, perc_diff_avgs))
        bestRes = min(erf_locdict.values())
        besterf_loc = [key for key in erf_locdict if erf_locdict[key] == bestRes]
        
        # We will repeat the above fit with this new 'optimized' split location
        fit_args = dual_tc_fit(big_data, path_to_plots[mat], erf_loc=min(besterf_loc), fit_orders=fit_orders, plots=False)
        perc_diff_avg, perc_diff_arr = get_percdiff(T, k, fit_args)
        print(f"Low-Hi split centered at : {min(besterf_loc)} ~~ with average percent difference value of: {perc_diff_avg:.2f}%")

        perc_diff_low, perc_diff_arr = get_percdiff(T[T<=max(low_fit_xs)],k[T<=max(low_fit_xs)], fit_args)
        perc_diff_hi, perc_diff_arr = get_percdiff(T[T>=min(hi_fit_xs)],k[T>=min(hi_fit_xs)], fit_args)
        perc_diff_combo, perc_diff_arr = get_percdiff(T,k, fit_args)
        fit_args["low_perc_err"] =  perc_diff_low
        fit_args["hi_perc_err"] =  perc_diff_hi
        fit_args["combined_perc_err"] =  perc_diff_combo

    if gaps:
        low_array = format_splitfit(fit_args, "low")
        high_array = format_splitfit(fit_args, "hi")

        create_data_table(low_array, f"{path_to_fits[mat]}\\{mat}_lo.txt")
        create_tc_csv(low_array, f"{path_to_fits[mat]}\\{mat}_lo.csv")
        create_data_table(high_array, f"{path_to_fits[mat]}\\{mat}_hi.txt")
        create_tc_csv(high_array, f"{path_to_fits[mat]}\\{mat}_hi.csv")
        make_fit_lh5(fit_args, path_to_fits[mat])

    else:
        output_array = format_combofit(fit_args)
        # Finally, we will output the fit parameters as a csv, and lh5 file - and plot the data.
        create_data_table(output_array, f"{path_to_fits[mat]}\\{mat}.txt")
        create_tc_csv(output_array, f"{path_to_fits[mat]}\\{mat}.csv")
        make_fit_lh5(fit_args, path_to_fits[mat])
        # PLOTTING CODE
    if plots:
        tk_plot(mat,path_to_RAW, data_dict, fit_args, fit_range = [100e-3, np.sort(T)[-1]], points=True, fits="combined", fill=True)

Aluminum_110 : Using a low fit
CFRP : Using a low fit
CFRP_Clearwater : Using a low fit
CFRP_DPP : Using a low fit
CFRP_Graphlite : Using a low fit
G10_FR4 : Using a low fit
Ketron : Using a low fit
Macor : Using a low fit
Stainless_Steel : Using a combined fit
Low-Hi split centered at : 119.47072857142858 ~~ with average percent difference value of: 6.15%


c:\Users\henac\OneDrive - The University of Texas at Austin\01_RESEARCH\05_CMBS4\Cryogenic_Material_Properties\thermal_conductivity\fit_types.py:29: RuntimeWarning: overflow encountered in power
  hi_fit = 10**np.polyval(hi_param, np.log10(T))
c:\Users\henac\OneDrive - The University of Texas at Austin\01_RESEARCH\05_CMBS4\Cryogenic_Material_Properties\thermal_conductivity\fit_types.py:41: RuntimeWarning: invalid value encountered in multiply
  k = low_fit*erf_low+hi_fit*erf_hi


Stainless_Steel_304 : Using a combined fit
Low-Hi split centered at : 119.7857 ~~ with average percent difference value of: 3.95%
